In [2]:
#!pip install scipy
import tensorflow as tf
import matplotlib.image as img
import numpy as np
import os
import matplotlib.pyplot as plt
import shutil
#from tensorflow.keras.preprocessing import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import scipy
#Image flow from directory, data image gen
#from skimage import io, color, filters
#from skimage.transform import resize, rotate
print(tf.__version__)
print(tf.test.gpu_device_name())
#!pip install tensorflow_datasets
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import os
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report


2.3.1



In [ ]:
labels_all= '''apple_pie
baby_back_ribs
baklava
beef_carpaccio
beef_tartare
beet_salad
beignets
bibimbap
bread_pudding
breakfast_burrito
bruschetta
caesar_salad
cannoli
caprese_salad
carrot_cake
ceviche
cheesecake
cheese_plate
chicken_curry
chicken_quesadilla
chicken_wings
chocolate_cake
chocolate_mousse
churros
clam_chowder
club_sandwich
crab_cakes
creme_brulee
croque_madame
cup_cakes
deviled_eggs
donuts
dumplings
edamame
eggs_benedict
escargots
falafel
filet_mignon
fish_and_chips
foie_gras
french_fries
french_onion_soup
french_toast
fried_calamari
fried_rice
frozen_yogurt
garlic_bread
gnocchi
greek_salad
grilled_cheese_sandwich
grilled_salmon
guacamole
gyoza
hamburger
hot_and_sour_soup
hot_dog
huevos_rancheros
hummus
ice_cream
lasagna
lobster_bisque
lobster_roll_sandwich
macaroni_and_cheese
macarons
miso_soup
mussels
nachos
omelette
onion_rings
oysters
pad_thai
paella
pancakes
panna_cotta
peking_duck
pho
pizza
pork_chop
poutine
prime_rib
pulled_pork_sandwich
ramen
ravioli
red_velvet_cake
risotto
samosa
sashimi
scallops
seaweed_salad
shrimp_and_grits
spaghetti_bolognese
spaghetti_carbonara
spring_rolls
steak
strawberry_shortcake
sushi
tacos
takoyaki
tiramisu
tuna_tartare
waffles'''
user_words = labels_all
word_list = user_words.split()
user_words = []
for word in word_list:
    user_words.append(word)
user_words = " ".join(user_words)
class_labels = user_words.split()

In [ ]:
# making only 5 classes

# possible agumentations (rotation_range=90, rescale=1./255,rotation_range=90,height_shift_range=0.5,shear_range=0.1, zoom_range=0.1, horizontal_flip=True))

X5_train_datagen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,shear_range=0.2, zoom_range=0.2)
X5_test_datagen = ImageDataGenerator(rescale=1./255)
X5_var_datagen = ImageDataGenerator(rescale=1./255)

labels_5 = ['apple_pie','waffles','gnocchi','chocolate_mousse','baklava']

X5_train = X5_train_datagen.flow_from_directory(
    '../data/train_mini/',
    target_size=(256,256),
    color_mode="rgb",
    classes=labels_5,
    class_mode="categorical",
    batch_size=32,
    shuffle=True,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    follow_links=False,
    subset=None,
    interpolation="nearest",
)

X5_test = X5_test_datagen.flow_from_directory(
    '../data/test_mini/',
    target_size=(256,256),
    color_mode="rgb",
    classes=labels_5,
    class_mode="categorical",
    batch_size=32,
    shuffle=False,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    follow_links=False,
    subset=None,
    interpolation="nearest",
)
X5_var = X5_var_datagen.flow_from_directory(
    '../data/var_mini/',
    target_size=(256,256),
    color_mode="rgb",
    classes=labels_5,
    class_mode="categorical",
    batch_size=32,
    shuffle=False,
    seed=None,
    save_to_dir=None,
    save_prefix="",
    save_format="png",
    follow_links=False,
    subset=None,
    interpolation="nearest",
)

print('These waffles have been: height_shift_range=0.5,shear_range=0.2, zoom_range=0.2, horizontal_flip=True')
from numpy import expand_dims
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from matplotlib import pyplot
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
# load the image
img = load_img('../data/train_mini/waffles/1005755.jpg')
# convert to numpy array
data = img_to_array(img)
# expand dimension to one sample
samples = expand_dims(data, 0)
# create image data augmentation generator
#datagen = ImageDataGenerator(height_shift_range=0.5)
# prepare iterator
it = X5_train_datagen.flow(samples, batch_size=1)
# generate samples and plot
pyplot.title(label='These waffles have bee height_shift_range=0.5,shear_range=0.2, zoom_range=0.2, horizontal_flip=True')
for i in range(9):
    # define subplot
    pyplot.subplot( 330 +1 +i)
    # generate batch of images
    batch = it.next()
    # convert to unsigned integers for viewing
    image = batch[0].astype('uint8')
    # plot raw pixel data
    pyplot.minorticks_off()
    pyplot.imshow(image)
    pyplot.axis('off')
    
# show the figure, works only when rescale 1/255 is off. 
pyplot.show()

In [ ]:
# Getting logs file ready for tensorboard
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True, monitor = 'val_loss') 
checkpoint_filepath = './CNN/custom_5classmodel.h5'
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


In [ ]:
tensorboard dev upload --logdir \
    './my_logs'

In [ ]:
def define_model(kernel_size=(3,3), input_shape=[256,256,3], pool_size=2, nb_classes=5):
    model = Sequential() 
    # options: 'linear', 'sigmoid', 'tanh', 'relu', 'softplus', 'softsign'
    model.add(Conv2D(16, (kernel_size[0], kernel_size[1]),
                        padding='same', 
                        input_shape=input_shape)) 
    model.add(Activation('relu')) 
    model.add(Conv2D(64, (kernel_size[0], kernel_size[1]), padding='same')) 
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size)) # decreases size, prevent overfitting
    model.add(Flatten()) # necessary to flatten before going into conventional dense layer
    print('Model flattened out to ', model.output_shape)
    # now start a neural network
    model.add(Dense(64))
    model.add(Activation('relu')) #64 neurons in this layer
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dropout(0.2)) # zeros out some fraction of inputs, helps prevent overfitting
    model.add(Dense(nb_classes)) # (one for each class)
    model.add(Activation('softmax')) # softmax at end to pick between classes 0-3 KEEP
    # optimizers to one of these: 'adam', 'adadelta', 'sgd'
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return model

model_complex = define_model()



history = model_complex.fit(
        X5_train,
        steps_per_epoch=3500//100,
        epochs=50,
        validation_data=X5_var,
        validation_steps=750//100)
#callbacks=[checkpoint_cb,early_stopping_cb,tensorboard_cb]

In [ ]:
model.predict(X_test)

In [ ]:

score = model.evaluate(X5_var, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1]) # this is the one we care about

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(50)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
X5_test.reset()
predIdxs = history.predict(x=X5_test,
    steps=(750 // 32) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(X5_test.classes, predIdxs,
    target_names=X5_test.class_indices.keys()))



In [ ]:
# loading saved model 
name = keras.models.load_model('./CNN/custom_5classmodel.h5')
predictions_ = name.predict(X5_test)


In [ ]:
print()
X5_test.reset()
predIdxs = modelw.predict(x=X5_test,
	steps=(750 // 32) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(X5_test.classes, predIdxs,
	target_names=X5_test.class_indices.keys()))


In [ ]:
def define_model2(kernel_size=(3,3), input_shape=[256,256,3], pool_size=3, nb_classes=5):
    model = Sequential() 
    model.add(Conv2D(16, (kernel_size[0], kernel_size[1]),
                        padding='same', 
                        input_shape=input_shape)) 
    model.add(Activation('relu')) 
    model.add(Conv2D(32, (kernel_size[0], kernel_size[1]), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size)) 
    model.add(Conv2D(64, (kernel_size[0], kernel_size[1]),
                        padding='same', 
                        input_shape=input_shape))
    model.add(Flatten()) # necessary to flatten before going into conventional dense layer  KEEP
    print('Model flattened out to ', model.output_shape)
    # now start a typical neural network
    model.add(Dense(128)) # (only) 32 neurons in this layer, really?   KEEP
    model.add(Activation('relu'))
#     model.add(Dropout(0.1)) # zeros out some fraction of inputs, helps prevent overfitting
    model.add(Dense(nb_classes)) # 5 final nodes (one for each class)  KEEP
    model.add(Activation('softmax')) # softmax at end to pick between classes 0-3 KEEP
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return model

model2 = define_model2()

#384,384

history2 = model2.fit(
        X5_train,
        steps_per_epoch=3500//32,
        epochs=15,
        validation_data=X5_var,
        validation_steps=750//32,
        callbacks=[checkpoint_cb,early_stopping_cb,tensorboard_cb])

In [ ]:
model2.summary()

In [ ]:
def define_model3(kernel_size=(3,3), input_shape=[256,256,3], pool_size=4, nb_classes=5):
    model = Sequential() 
    # options: 'linear', 'sigmoid', 'tanh', 'relu', 'softplus', 'softsign'
    model.add(Conv2D(16, (kernel_size[0], kernel_size[1]),
                        padding='same', strides=1,
                        input_shape=input_shape)) 
    model.add(Activation('relu')) # Activation specification necessary for Conv2D and Dense layers
    model.add(Conv2D(16, (kernel_size[0], kernel_size[1]), padding='same')) 
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size)) # decreases size, helps prevent overfitting
    model.add(Conv2D(64, (kernel_size[0], kernel_size[1]),
                        padding='same', strides=1,
                        input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (kernel_size[0], kernel_size[1]),
                        padding='same', strides=1,
                        input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Flatten()) # necessary to flatten before going into conventional dense layer
    print('Model flattened out to ', model.output_shape)
    # now start a typical neural network
    model.add(Dense(64))   
    model.add(Activation('relu'))
    model.add(Dense(nb_classes)) # 5 final nodes (one for each class) 
    model.add(Activation('softmax')) 
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return model

model3 = define_model3()


history3 = model3.fit(
        X5_train,
        steps_per_epoch=3500//32,
        epochs=50,
        validation_data=X5_var,
        validation_steps=750//32)

In [ ]:
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Training Accuracy","Validation Accuracy","Training loss","Validation Loss"])
plt.show()

 
X5_test.reset()
predIdxs = model_on_.predict(x=X5_test,
    steps=(750 // 32) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(X5_test.classes, predIdxs,
    target_names=X5_test.class_indices.keys()))



# Ran out of power must SSH into some power on AWS